In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
img_150_list_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/final_pj/merged_food_by_Mook.csv")
recipe_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/final_pj/recipe_df_clean.csv")

In [5]:
img_150_list_df.head(2)

,name,ingredients,recipe
0,갈비구이,"닭가슴살 200g,대파 1개,간장 2T,설탕 1T,참기름 0.5T,다진마늘 0.5T...","1. 간장2T, 설탕1T, 참기름0.5T, 다진마늘0.5T, 꿀1T를 넣고 갈비양념..."
1,갈치구이,"갈치 2마리,식용유 3T,부침가루 2T,쌀뜨물 적당량,후추 약간,맛소금 2꼬집",1. 갈치를 쌀뜨물에 넣어 해동 해 줍니다. 생물은 그냥 조리하셔도 됩니다.|2. ...


In [6]:
recipe_df.head(2)

,CKG_NM,INQ_CNT,RCMM_CNT,SRAP_CNT,CKG_MTH_ACTO_NM,CKG_MTRL_ACTO_NM,CKG_KND_ACTO_NM,CKG_MTRL_CN,vegetarian_type
0,소고기떡국,743,0,2,끓이기,소고기,국/탕,['재료 떡국떡400g 다진소고기100g 멸치육수800ml 대파13대 계란2개 참기...,Omnivore
1,된장수육,1396,0,1,삶기,돼지고기,메인반찬,['재료 돼지고기 수육용삼겹살500g 된장15큰술 술4큰술 홍어무침 무생채 콩나물무침'],Omnivore


In [7]:
recipe_df.columns

Index(['CKG_NM', 'INQ_CNT', 'RCMM_CNT', 'SRAP_CNT', 'CKG_MTH_ACTO_NM',
       'CKG_MTRL_ACTO_NM', 'CKG_KND_ACTO_NM', 'CKG_MTRL_CN',
       'vegetarian_type'],
      dtype='object')

In [21]:
# img_150_list_df에 있지만 recipe_df에 없는 음식 이름 찾기
missing_names = img_150_list_df[~img_150_list_df['name'].isin(recipe_df['CKG_NM'])]

# 결과 확인
print(missing_names["name"])


3         곱창구이
14      떡국_만두국
18        시래기국
21         과매기
23          젓갈
27     후라이드_치킨
44          경단
45          꿀떡
48        조개구이
67        꽁치조림
75         닭계장
81     곰탕(설렁탕)
95          약과
96          약식
97          한과
98          멍게
101        산낙지
Name: name, dtype: object


In [23]:
# !pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.4 MB/s eta 0:00:00


In [26]:
from rapidfuzz import fuzz, process

# 누락된 음식 이름 리스트
missing_names = [
    "곱창구이", "떡국_만두국", "시래기국", "과매기", "젓갈", "후라이드_치킨",
    "경단", "꿀떡", "조개구이", "꽁치조림", "닭계장", "곰탕(설렁탕)",
    "약과", "약식", "한과", "멍게", "산낙지"
]

# 레시피 데이터프레임에서 음식 이름 목록 추출
recipe_names = recipe_df['CKG_NM'].tolist()

# 누락된 음식 이름과 레시피 이름 간의 유사도 계산
similar_names = {}
for missing in missing_names:
    matches = process.extract(missing, recipe_names, scorer=fuzz.token_sort_ratio, limit=5)
    similar_names[missing] = matches

# 결과 출력
for missing, matches in similar_names.items():
    print(f"'{missing}'와 유사한 이름들:")
    for match, score, _ in matches:
        print(f"  - {match} (유사도: {score}%)")
    print()


'곱창구이'와 유사한 이름들:
  - 들기름곱창김구이 (유사도: 66.66666666666667%)
  - 귤구이 (유사도: 57.14285714285714%)
  - 김구이 (유사도: 57.14285714285714%)
  - 굴구이 (유사도: 57.14285714285714%)
  - 곱창김 (유사도: 57.14285714285714%)

'떡국_만두국'와 유사한 이름들:
  - 떡만두국 (유사도: 80.0%)
  - 김떡만두국 (유사도: 72.72727272727273%)
  - 사골떡만두국 (유사도: 66.66666666666667%)
  - 만두국 (유사도: 66.66666666666667%)
  - 계란떡만두국 (유사도: 66.66666666666667%)

'시래기국'와 유사한 이름들:
  - 시래기 (유사도: 85.71428571428572%)
  - 들깨시래기국 (유사도: 80.0%)
  - 시래기순대국 (유사도: 80.0%)
  - 시래기된장국 (유사도: 80.0%)
  - 꼬시래기국수 (유사도: 80.0%)

'과매기'와 유사한 이름들:
  - 과메기 (유사도: 66.66666666666667%)
  - 사과깍두기 (유사도: 50.0%)
  - 매콤조기조림 (유사도: 44.44444444444444%)
  - 매콤닭불고기 (유사도: 44.44444444444444%)
  - 용과 (유사도: 40.0%)

'젓갈'와 유사한 이름들:
  - 광어젓갈 (유사도: 66.66666666666667%)
  - 멍게젓갈 (유사도: 66.66666666666667%)
  - 오징어젓갈 (유사도: 57.14285714285714%)
  - 낙지젓갈무침 (유사도: 50.0%)
  - 돼지젓갈조림 (유사도: 50.0%)

'후라이드_치킨'와 유사한 이름들:
  - 후라이드치킨 (유사도: 92.3076923076923%)
  - 닭가슴살순살후라이드치킨 (유사도: 63.1578947368421%)
  - 계란후라이샌드위치 (유사도: 62.5%)
  - 돼지후라이드

In [34]:
# img_150_list_df의 이름 기준으로 recipe_df의 이름 변경 매핑
# "recipe_df의 기존 음식 이름": "img_150_list_df의 기준 음식 이름"
name_mapping_ = {
    "떡만두국": "떡국_만두국",
    "시래기된장국": "시래기국",
    "과메기": "과매기",
    "설렁탕": "곰탕(설렁탕)",
    "오징어젓갈": "젓갈",
    "후라이드치킨": "후라이드_치킨",
    "카스테라경단": "경단",
    "꽁치무조림": "꽁치조림",
    "닭개장": "닭계장",
    "약밥": "약식",
    "멍게무침": "멍게",
    "낙지탕탕이": "산낙지",
    # 아래부터는 비슷한 이름이 없어서 눈으로 찾아 바꿈
    # "": "곱창구이",
    # "": "꿀떡",
    # "": "조개구이",
    # "": "약과",
    # "": "한과",
}

In [36]:
# 'name' 열을 매핑 결과로 업데이트
recipe_df['CKG_NM'] = recipe_df['CKG_NM'].replace(name_mapping)

# 결과 확인
print(recipe_df[recipe_df['CKG_NM'] == '닭계장'])
print(recipe_df[recipe_df['CKG_NM'] == '꽁치조림'])
print(img_150_list_df[img_150_list_df['name'] == '닭계장'])
print(img_150_list_df[img_150_list_df['name'] == '꽁치조림'])

    CKG_NM  INQ_CNT  RCMM_CNT  SRAP_CNT CKG_MTH_ACTO_NM CKG_MTRL_ACTO_NM  \
206    닭계장      481         0         4             끓이기              닭고기   

    CKG_KND_ACTO_NM                                        CKG_MTRL_CN  \
206             국/탕  ['재료 60인분 닭3kg 무2kg 대파8개 통마늘20개 통후추30알 양념 60인분...   

    vegetarian_type  
206        Omnivore  
     CKG_NM  INQ_CNT  RCMM_CNT  SRAP_CNT CKG_MTH_ACTO_NM CKG_MTRL_ACTO_NM  \
1541   꽁치조림      741         0         1              조림              해물류   

     CKG_KND_ACTO_NM                                        CKG_MTRL_CN  \
1541              찌개  ['재료 꽁치통조림큰거 무우 쌀뜨물300ml 양파12개 대파 고추 간장2숟가락 양념...   

     vegetarian_type  
1541     Pescatarian  
   name                                        ingredients  \
75  닭계장  닭 1마리,고사리 200g,숙주나물 1봉지,대파 3뿌리,양파 1개,홍고추 3개,물 ...   

                                               recipe  
75  1. 먼저 마트에 흔히 파는 닭한마리 준비해주세요.  촌닭으로 하셔도 무방하지만 그...  
    name                                        ingredients  \
67

In [38]:
# 변경된 이름과 img_150_list_df의 이름 매칭 여부 확인
matched_names = recipe_df[recipe_df['CKG_NM'].isin(img_150_list_df['name'])]
unmatched_names = recipe_df[~recipe_df['CKG_NM'].isin(img_150_list_df['name'])]

print(f"매칭된 이름 수: {len(matched_names)}")
print(f"매칭되지 않은 이름들: {unmatched_names}")


매칭된 이름 수: 144
매칭되지 않은 이름들:          CKG_NM  INQ_CNT  RCMM_CNT  SRAP_CNT CKG_MTH_ACTO_NM CKG_MTRL_ACTO_NM  \
0         소고기떡국      743         0         2             끓이기              소고기   
1          된장수육     1396         0         1              삶기             돼지고기   
2        우거지감자탕     4008         0        29             끓이기             돼지고기   
3          만두전골     6350         0         6             끓이기            가공식품류   
4         무수분보쌈     1829         0         6              삶기             돼지고기   
...         ...      ...       ...       ...             ...              ...   
11272   새우양배추볶음       27         0         2              볶음              해물류   
11273  북어채미나리무침       31         0         3              무침              채소류   
11274    백조기생선찜       10         0         0               찜              해물류   
11275    초코곤약모찌       10         0         0              굽기               기타   
11276        묵밥        8         0         0             끓이기               기타   

